# Evaluate Playlist generation / augmentation

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, time, gzip
import pickle as pkl
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from scipy.sparse import lil_matrix, issparse

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from models import MTC
from tools import calc_RPrecision_HitRate

In [ ]:
TOPs = [5, 10, 20, 30, 50, 100, 200, 300, 500, 1000]

In [ ]:
datasets = ['aotm2011', '30music']

In [ ]:
dix = 0
dataset_name = datasets[dix]
dataset_name

In [ ]:
# X = pkl.load(gzip.open(os.path.join(base_dir, 'X_train_dev.pkl.gz'), 'rb'))
# Y = pkl.load(gzip.open(os.path.join(base_dir, 'Y.pkl.gz'), 'rb'))
# PU_test = pkl.load(gzip.open(os.path.join(base_dir, 'PU_test.pkl.gz'), 'rb'))
# playlists2 = pkl.load(gzip.open(os.path.join(base_dir, 'playlists_train_dev_test_s2.pkl.gz'), 'rb'))

In [ ]:
n_seed = 4
data_dir = 'data/%s/setting2' % dataset_name
X = pkl.load(gzip.open(os.path.join(data_dir, 'X_trndev_%d.pkl.gz' % n_seed), 'rb'))
Y = pkl.load(gzip.open(os.path.join(data_dir, 'Y.pkl.gz'), 'rb'))
PU_test = pkl.load(gzip.open(os.path.join(data_dir, 'PU_test_%d.pkl.gz' % n_seed), 'rb'))
song2pop = pkl.load(gzip.open(os.path.join(data_dir, 'song2pop.pkl.gz'), 'rb'))
playlists2 = pkl.load(gzip.open(os.path.join(data_dir, 'playlists_train_dev_test_s2_%d.pkl.gz' % n_seed), 'rb'))

In [ ]:
Y_test = Y[:, -PU_test.shape[1]:]
print(Y_test.shape)
#Y_test.sum(axis=0)

### Popularity based recommendation

In [ ]:
all_songs = pkl.load(gzip.open(os.path.join(data_dir, 'all_songs.pkl.gz'), 'rb'))
index2song = {ix: sid for ix, (sid, _) in enumerate(all_songs)}

In [ ]:
song2pop_test = song2pop.copy()
for ppl in playlists2['test_playlists_held']:
    for sid in ppl:
        song2pop_test[sid] -= 1

In [ ]:
rps_pop = []
hitrates_pop = {top: [] for top in TOPs}

assert Y_test.shape == PU_test.shape
for j in range(Y_test.shape[1]):
    if (j+1) % 10 == 0:
        sys.stdout.write('\r%d / %d' % (j+1, Y_test.shape[1]))
        sys.stdout.flush()
    y1 = Y_test[:, j].toarray().reshape(-1)
    y2 = PU_test[:, j].toarray().reshape(-1)
    indices = np.where(0 == y2)[0]
    y_true = y1[indices]
    y_pred = np.array([song2pop_test[index2song[ix]] for ix in indices])
    
    rp, hr_dict = calc_RPrecision_HitRate(y_true, y_pred, tops=TOPs)
    rps_pop.append(rp)
    for top in TOPs:
        hitrates_pop[top].append(hr_dict[top])
print('\n%d / %d' % (len(rps_pop), PU_test.shape[1]))

In [ ]:
ax = plt.subplot(111)
ax.hist(rps_pop, bins=100)
ax.set_yscale('log')
#ax.set_xlim(0, xmax)
pass

In [ ]:
pop_perf = {dataset_name: {'Test': {'R-Precision': np.mean(rps_pop), 
                                    'Hit-Rate': {top: np.mean(hitrates_pop[top]) for top in hitrates_pop}}}}
pop_perf

In [ ]:
fperf_pop = os.path.join(data_dir, 'perf-pop-%d.pkl' % n_seed)
print(fperf_pop)
pkl.dump(pop_perf, open(fperf_pop, 'wb'))
pkl.load(open(fperf_pop, 'rb'))